# microsoftmt
> Library to use Microsoft's machine translation services

In [ ]:
#| default_exp microsoftmt

In [ ]:
#| hide
import os
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    homedir = "/content/drive/MyDrive"
else:
    homedir = os.getenv('HOME')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#| hide
if running_in_colab:
    github_test_folder = homedir+"/github/polyglottech/mteval"
    %cd {github_test_folder}
    !pip3 install nbdev
    !pip3 install -e '.[dev]'

/content/drive/MyDrive/github/polyglottech/mteval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/github/polyglottech/mteval
  Attempting uninstall: mteval
    Found existing installation: mteval 0.0.1
    Can't uninstall 'mteval'. No files were found to uninstall.
  Running setup.py develop for mteval


In [ ]:
#| hide
from dotenv import load_dotenv

if running_in_colab:
    # Colab doesn't have a mechanism to set environment variables other than python-dotenv
    env_file = homedir+'/secrets/.env'
    %load_ext dotenv
    %dotenv {env_file}

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests, uuid
import os

class microsofttranslate:
    """
    Class to get translations from the Microsoft Translator API
    """
    def __init__(self):
        """Constructor of microsofttranslate class"""
        self._subscription_key = os.getenv('MS_SUBSCRIPTION_KEY')
        self._region = os.getenv('MS_REGION')

    def translate_text(self,source, target, text):
        """Function to translate text into the target language        """
        # Add your subscription key and endpoint
        subscription_key = self._subscription_key
        endpoint = "https://api.cognitive.microsofttranslator.com"

        # Add your location, also known as region. The default is global.
        # This is required if using a Cognitive Services resource.
        location = self._region

        path = '/translate'
        constructed_url = endpoint + path

        params = {
            'api-version': '3.0',
            'from': source,
            'to': [target]
        }
        constructed_url = endpoint + path

        headers = {
            'Ocp-Apim-Subscription-Key': subscription_key,
            'Ocp-Apim-Subscription-Region': location,
            'Content-type': 'application/json',
            'X-ClientTraceId': str(uuid.uuid4())
        }

        # You can pass more than one object in body.
        body = [{
            'text': text
        }]

        request = requests.post(constructed_url, params=params, headers=headers, json=body)
        response = request.json()

        return response[0]["translations"][0]["text"]

In [ ]:
#| hide
msmt = microsofttranslate()
translation = msmt.translate_text("en","it","This is a cool car!")
assert translation == "Questa è una macchina fantastica!"

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()